In [1]:
# import lib
import numpy as np
import pandas as pd
import os
import re

from colorama import Fore, Style
from sklearn.base import clone, BaseEstimator, RegressorMixin
from sklearn.metrics import cohen_kappa_score, accuracy_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim

from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline

import random


# 1. Initial setup and seed
setup params of LightGBM, XGBoost, CatBoost

In [2]:
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
SEED = 42
n_splits = 5

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(2024)

#Basic_demos_sex, cgas-cgas_score, max_stage, endurance-time_mins, fgc-fgc_srl_zone

train_featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'sii', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
                'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
                'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW', 'BMI_PHR']

test_featuresCols = ['Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-CGAS_Score', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T',
                'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age','Internet_Hours_Age','BMI_Internet_Hours',
                'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
                'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW', 'BMI_PHR']


Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01,  # Increased from 2.68e-06
    'device': 'cpu',
}


XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': SEED,
    'tree_method': 'gpu_hist'
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10,  # Increase this value
    'task_type': 'GPU',
}


In [3]:
# functions

# thong ke parquet file
def pre_progress_parquet_data(file_name, parquet_dir):
    file_path = os.path.join(parquet_dir, file_name, 'part-0.parquet')
    df = pd.read_parquet(file_path)
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), file_name.split('=')[1]


# xu li tat ca parquet song song
# tra ve dfchua cac stat va id
def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: pre_progress_parquet_data(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df


**preprocess_feature**: tao cac feature moi dua tren cac feature hien co

**auto_encoder**: ma hoa va giam chieu du lieu

In [4]:
def preprocess_feature(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    df['BMI_PHR'] = df['Physical-BMI'] * df['Physical-HeartRate']
    
    return df

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.LeakyReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2+10),
            nn.LeakyReLU(),
            nn.Linear(encoding_dim*2+10, encoding_dim),
            nn.LeakyReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim+15),
            nn.LeakyReLU(),
            nn.Linear(encoding_dim+15, encoding_dim*3),
            nn.LeakyReLU(),
            nn.Linear(encoding_dim*3, input_dim),
            nn.Sigmoid()
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# encoder du lieu
def auto_encoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded
        

In [5]:
# data loader
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
#test = train.head(20)

import pandas as pd
import numpy as np

# Giả sử `df` là bảng dữ liệu
train_non_id = train.drop("id", axis=1)
train_non_id = preprocess_feature(train_non_id)

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")
#test_ts = train_ts.head(20)
df_train = train_ts.drop('id', axis=1) 
df_test = test_ts.drop('id', axis=1) # bo cot id


encoded_df_train = auto_encoder(df_train)
encoded_df_test = auto_encoder(df_test)

# chuyen ve dang dataframe
train_ts_encoded = pd.DataFrame(encoded_df_train, columns=[f'Enc_{i + 1}' for i in range(encoded_df_train.shape[1])])
test_ts_encoded = pd.DataFrame(encoded_df_test, columns=[f'Enc_{i + 1}' for i in range(encoded_df_test.shape[1])])

time_series_cols = train_ts_encoded.columns.tolist()
train_ts_encoded["id"]=train_ts["id"]
test_ts_encoded['id']=test_ts["id"]

train = pd.merge(train, train_ts_encoded, how="left", on='id')
test = pd.merge(test, test_ts_encoded, how="left", on='id')

100%|██████████| 2/2 [00:00<00:00, 11.42it/s]


Epoch [10/50], Loss: 1.5837]
Epoch [20/50], Loss: 1.5409]
Epoch [30/50], Loss: 1.5265]
Epoch [40/50], Loss: 1.5112]
Epoch [50/50], Loss: 1.5038]
Epoch [10/50], Loss: 1.0764]
Epoch [20/50], Loss: 0.8212]
Epoch [30/50], Loss: 0.4822]
Epoch [40/50], Loss: 0.4271]
Epoch [50/50], Loss: 0.4271]


In [6]:
# correlation_matrix = train_non_id.corr()

# # Vẽ biểu đồ heatmap (tùy chọn)
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(40, 40))  # Thay đổi kích thước (20x20 là ví dụ)
# sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True)

# plt.title("Correlation Matrix", fontsize=16)
# plt.show()

In [7]:
# imputer de fill gia tri nan

train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer = KNNImputer(n_neighbors=6)

numeric_cols_train = train.select_dtypes(include=['int32', 'int64', 'float64']).columns
numeric_cols_test = test.select_dtypes(include=['int32', 'int64', 'float64']).columns

imputed_train_data = imputer.fit_transform(train[numeric_cols_train])
imputed_test_data = imputer.fit_transform(test[numeric_cols_test])

train_imputed = pd.DataFrame(imputed_train_data, columns=numeric_cols_train)
test_imputed = pd.DataFrame(imputed_test_data, columns=numeric_cols_test)

train_imputed['sii'] = train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols_train:
        train_imputed[col] = train[col]

for col in test.columns:
    if col not in numeric_cols_test:
        test_imputed[col] = test[col]

train = train_imputed
test = test_imputed

train = preprocess_feature(train)
train = train.dropna(thresh=10, axis=0)
test = preprocess_feature(test)

In [8]:
train_featuresCols += time_series_cols

train = train[train_featuresCols]
# drop cac ban ghi co sii nan
train = train.dropna(subset='sii') 

test_featuresCols += time_series_cols
test = test[test_featuresCols]

# con cai nao to qua thi nan
if np.any(np.isinf(train)):
    train = train.replace([np.inf, -np.inf], np.nan)

if np.any(np.isinf(test)):
    test = test.replace([np.inf, -np.inf], np.nan)

In [9]:
def eval_models(thresholds, y, y_predict_non_rounded):
    y_predic = np.where(y_predict_non_rounded < thresholds[0], 0,
                    np.where(y_predict_non_rounded < thresholds[1], 1,
                             np.where(y_predict_non_rounded < thresholds[2], 2, 3)))
    return -cohen_kappa_score(y, y_predic, weights='quadratic')

def train_models(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        y_train_pred_rounded = y_train_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = cohen_kappa_score(y_train, y_train_pred_rounded, weights='quadratic')        
        val_kappa = cohen_kappa_score(y_val, y_val_pred_rounded, weights='quadratic')

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(eval_models,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = np.where(oof_non_rounded < (KappaOPtimizer.x)[0], 0,
                    np.where(oof_non_rounded < (KappaOPtimizer.x)[1], 1,
                             np.where(oof_non_rounded < (KappaOPtimizer.x)[2], 2, 3)))
    
    tKappa = cohen_kappa_score(y, oof_tuned, weights='quadratic')

    print(f"----> || Optimized SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = np.where(tpm < (KappaOPtimizer.x)[0], 0,
                    np.where(tpm < (KappaOPtimizer.x)[1], 1,
                             np.where(tpm < (KappaOPtimizer.x)[2], 2, 3)))
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

In [10]:
# loading model
Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
],weights=[4.0,4.0,6.0])

In [11]:
train

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW,BMI_PHR,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_12,Enc_13,Enc_14,Enc_15,Enc_16,Enc_17,Enc_18,Enc_19,Enc_20,Enc_21,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_49,Enc_50
0,5.0,0.0,51.000000,16.877316,46.000000,50.800000,23.000000,62.166667,84.500000,107.666667,0.0,0.000000,0.000000,0.000000,7.000000,0.000000,6.000000,0.000000,6.000000,1.000000,2.000000,2.668550,16.879200,932.498000,1492.000000,8.255980,41.586200,13.817700,3.061430,9.213770,1.000000,24.434900,8.895360,38.917700,19.541300,32.690900,46.166667,60.166667,3.0,2,84.386578,15.0,50.631947,0.545865,1.499679,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453,1426.133176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,0.0,65.000000,14.035590,48.000000,46.000000,22.000000,75.000000,70.000000,122.000000,3.0,0.000000,5.000000,0.000000,11.000000,1.000000,11.000000,1.000000,3.000000,0.000000,2.000000,2.579490,14.037100,936.656000,1498.650000,6.019930,42.029100,12.825400,1.211720,3.970850,1.000000,21.035200,14.974000,39.449700,15.410700,27.055200,46.000000,64.000000,0.0,0,126.320313,0.0,0.000000,0.282883,3.229888,0.305154,1.458119,3719.320478,5950.914352,20.362087,32.579348,0.321056,12.718037,0.588157,0.777492,982.491320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,1.0,71.000000,16.648696,56.500000,75.600000,25.000000,65.000000,94.000000,117.000000,20.0,1.000000,7.000000,1.000000,10.000000,1.000000,10.000000,1.000000,5.000000,0.000000,2.666667,3.324367,18.689850,1080.290500,1843.476667,15.851497,57.328083,14.669483,4.020353,16.171908,2.666667,28.177517,13.299080,54.003700,26.326717,44.029000,38.000000,54.000000,2.0,0,166.486961,20.0,33.297392,0.865278,0.907097,0.248601,1.226548,17470.358939,29812.535668,14.289557,24.384612,0.465960,6.548359,0.582394,0.639976,1564.977430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,0.0,71.000000,18.292347,56.000000,81.600000,25.666667,60.000000,97.000000,117.000000,18.0,1.000000,5.000000,0.000000,7.000000,0.000000,7.000000,0.000000,7.000000,1.000000,3.000000,3.841910,18.294300,1131.430000,1923.440000,15.592500,62.775700,14.074000,4.220330,18.824300,2.000000,30.404100,16.779000,58.933800,26.479800,45.996600,31.000000,45.000000,0.0,1,164.631122,0.0,0.000000,1.028971,0.747651,0.224196,1.281264,21298.377749,36207.411592,13.865564,23.571569,0.472854,6.274343,0.563684,0.661008,1774.357653,8.329063,1.990211,7.612260,7.601535,9.108830,4.181010,5.54880,-0.075382,8.268166,6.624942,10.878230,3.629679,5.432630,-0.050110,5.599047,8.823846,4.702280,4.612934,-0.042706,8.717288,0.480118,4.182630,5.31

In [12]:
test

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,BMI_Age,Internet_Hours_Age,BMI_Internet_Hours,BFP_BMI,FFMI_BFP,FMI_BFP,LST_TBW,BFP_BMR,BFP_DEE,BMR_Weight,DEE_Weight,SMM_Height,Muscle_to_Fat,Hydration_Status,ICW_TBW,BMI_PHR,Enc_1,Enc_2,Enc_3,Enc_4,Enc_5,Enc_6,Enc_7,Enc_8,Enc_9,Enc_10,Enc_11,Enc_12,Enc_13,Enc_14,Enc_15,Enc_16,Enc_17,Enc_18,Enc_19,Enc_20,Enc_21,Enc_22,Enc_23,Enc_24,Enc_25,Enc_26,Enc_27,Enc_28,Enc_29,Enc_30,Enc_31,Enc_32,Enc_33,Enc_34,Enc_35,Enc_36,Enc_37,Enc_38,Enc_39,Enc_40,Enc_41,Enc_42,Enc_43,Enc_44,Enc_45,Enc_46,Enc_47,Enc_48,Enc_49,Enc_50
0,5.0,0.0,51.000000,16.877316,46.000000,50.800000,25.4,66.500000,80.500000,107.500000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,6.000000,0.000000,6.000000,1.0,2.000000,2.668550,16.879200,932.498000,1492.000000,8.255980,41.586200,13.817700,3.061430,9.213770,1.000000,24.434900,8.895360,38.917700,19.541300,32.690900,39.500000,55.833333,3.000000,84.386578,15.000000,50.631947,0.545865,1.499679,0.332267,1.190475,8591.822097,13746.944840,18.356260,29.370079,0.424811,6.383063,0.643522,0.747453,1358.623913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,0.0,63.166667,14.035590,48.000000,46.000000,22.0,75.000000,70.000000,122.000000,3.000000,0.000000,5.000000,0.000000,11.000000,1.000000,11.000000,1.000000,3.000000,0.0,2.000000,2.579490,14.037100,936.656000,1498.650000,6.019930,42.029100,12.825400,1.211720,3.970850,1.000000,21.035200,14.974000,39.449700,15.410700,27.055200,46.000000,64.000000,0.000000,126.320313,0.000000,0.000000,0.282883,3.229888,0.305154,1.458119,3719.320478,5950.914352,20.362087,32.579348,0.321056,12.718037,0.588157,0.777492,982.491320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,1.0,71.000000,16.648696,56.500000,75.600000,25.4,65.000000,94.000000,117.000000,20.000000,1.000000,7.000000,1.000000,10.000000,1.000000,10.000000,1.000000,5.000000,0.0,2.833333,3.496302,17.404817,1070.525667,1880.245000,14.484752,56.288233,14.012633,3.392192,14.178437,1.500000,27.347717,14.455743,52.791950,24.385267,41.832500,38.000000,54.000000,2.000000,166.486961,20.000000,33.297392,0.814627,0.988306,0.239250,1.261984,15178.380365,26658.934650,14.160392,24.870966,0.431598,7.188646,0.553340,0.653743,1564.977430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,0.0,71.000000,18.292347,56.000000,81.600000,25.4,60.000000,97.000000,117.000000,18.000000,1.000000,5.000000,0.000000,7.000000,0.000000,7.000000,0.000000,7.000000,1.0,3.000000,3.841910,18.294300,1131.430000,1923.440000,15.592500,62.775700,14.074000,4.220330,18.824300,2.000000,30.404100,16.779000,58.933800,26.479800,45.996600,31.000000,45.000000,0.000000,164.631122,0.000000,0.000000,1.028971,0.747651,0.224196,1.281264,21298.377749,36207.411592,13.865564,23.571569,0.472854,6.274343,0.563684,0.661008,1774.357653,-0.010104,15.968156,0.961367,-0.021360,11.445395,-0.075853,7.942317,6.920491,11.948594,-0.104204,15.360100,1.863419,9.896029,8.360224,16.795908,-0.090223,-0.088060,-0.103157,-0.049872,0.154920,0.75

In [13]:
# train
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')
Submission1 = train_models(voting_model, test)

# Save submission
Submission1.to_csv('submission.csv', index=False)

Training Folds: 100%|██████████| 5/5 [00:27<00:00,  5.57s/it]

Mean Train QWK --> 0.7701
Mean Validation QWK ---> 0.4857
----> || Optimized SCORE ::  0.540


In [14]:
Submission1

,id,sii
0,00008ff9,1
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,1
7,0068a485,0
8,0069fbed,1
9,0083e397,1


In [15]:
Submission1['sii'].value_counts()

sii
1    10
0    10
Name: count, dtype: int64